# Algoritmos Genéticos

In [57]:
import random
import numpy as np
import pandas as pd

### Fichas

In [4]:
fichas = {
    "a": 10,
    "b": 3,
    "c": 7,
    "d": 1,
    "e": 2,
    "f": 8,
    "g": 4,
    "h": 10,
    "i": 9,
    "j": 3
}

## Clase Individuos

- **generate_random_array:** generación de cada individuo con **a** y **j** en la primera y última posición, generándose de forma aleatoria los otros 6 valores para tener cada individuo con 8 cromosomas
- **fitness_function:** aplicación de la **función de calidad** a cada cromosoma del individuo y se aplica la función inversa a cada uno (Se realiza una multiplicación con los adyacentes)
- **function_adition:** suma de la función y la función inversa de cada cromosoma para obtener el total correspondiente al individuo

In [15]:
class Individuos():
    def __init__(self, fichas):
        self.fichas = fichas
        self.individuo = []
        self.function = []
        self.inverse_function = []
        self.total_function = 0
        self.total_inverse_function = 0
        self.probabilidad = 0
        self.probabilidad_acumulada = []

    def generate_random_array(self):
        random_array = [random.randint(2, 9) for _ in range(6)]
        self.individuo.append(fichas[list(fichas.keys())[0]])
        for i in range(6):
            self.individuo.append(fichas[random.choice(list(fichas.keys()))])
        self.individuo.append(fichas[list(fichas.keys())[-1]])
    
    def fitness_function(self):
        for i in range(len(self.individuo)):
            if i == 0:
                self.function.append(self.individuo[i] * self.individuo[i + 1])
            elif i == len(self.individuo) - 1:
                self.function.append(self.individuo[i] * self.individuo[i - 1])
            else:
                self.function.append(self.individuo[i] * self.individuo[i - 1] * self.individuo[i + 1])
            self.inverse_function.append(1 / self.function[i])
        
        for value in self.function:
            self.inverse_function.append(1 / value)
    
    def function_addition(self):
        for i in range(len(self.function)):
            self.total_function += self.function[i]
        self.total_inverse_function = 1/self.total_function   

### Implementación

In [5]:
individuo = Individuos(fichas)
individuo.generate_random_array()
individuo.fitness_function()
individuo.function_addition()

print("Individuo:", individuo.individuo)
print("Fitness Function:", individuo.function)
print("Inverse Function:", individuo.inverse_function)
print("Total Function:", individuo.total_function)
print("Total Inverse Function:", individuo.total_inverse_function)

Individuo: [10, 3, 4, 4, 2, 4, 1, 3]
Fitness Function: [30, 120, 48, 32, 32, 8, 12, 3]
Inverse Function: [0.03333333333333333, 0.008333333333333333, 0.020833333333333332, 0.03125, 0.03125, 0.125, 0.08333333333333333, 0.3333333333333333, 0.03333333333333333, 0.008333333333333333, 0.020833333333333332, 0.03125, 0.03125, 0.125, 0.08333333333333333, 0.3333333333333333]
Total Function: 285
Total Inverse Function: 0.0035087719298245615


## Creación de población y probabilidades

- **creacion_poblacion:** se genera una población (array) con la cantidad de individuos que se asignen
- **probabilidades:** se calcula la probabilidad (total) de cada individuo, utilizando (1/f)/sum(f)

In [6]:
def creacion_poblacion(fichas, cantidad):
    poblacion = []
    for _ in range(cantidad):
        individuo = Individuos(fichas)
        individuo.generate_random_array()
        individuo.fitness_function()
        individuo.function_addition()
        poblacion.append(individuo)
    return poblacion

def probabilidades(poblacion):
    for individuo in poblacion:
        individuo.probabilidad = individuo.total_inverse_function / sum(ind.total_inverse_function for ind in poblacion)

def probabilidades_acum(poblacion):
    acumulada = 0
    for individuo in poblacion:
        acumulada += individuo.probabilidad
        individuo.probabilidad_acumulada.append(acumulada)

### Implementación

In [7]:
poblacion = creacion_poblacion(fichas, 5)
probabilidades(poblacion)
probabilidades_acum(poblacion)

print("Probabilidades Acumuladas:")
for ind in poblacion:
    print(f"Individuo: {ind.individuo}, Funcion: {ind.total_function}, Probabilidad: {ind.probabilidad}, Probabilidad Acumulada: {ind.probabilidad_acumulada}")

print("Total Probabilidad:", sum(ind.probabilidad for ind in poblacion))

Probabilidades Acumuladas:
Individuo: [10, 10, 10, 8, 4, 7, 9, 3], Funcion: 2912, Probabilidad: 0.04617513281464231, Probabilidad Acumulada: [0.04617513281464231]
Individuo: [10, 1, 3, 3, 10, 9, 1, 3], Funcion: 529, Probabilidad: 0.25418144944468507, Probabilidad Acumulada: [0.3003565822593274]
Individuo: [10, 10, 1, 2, 7, 3, 10, 3], Funcion: 606, Probabilidad: 0.22188446659445282, Probabilidad Acumulada: [0.5222410488537802]
Individuo: [10, 1, 7, 2, 4, 7, 4, 3], Funcion: 414, Probabilidad: 0.32478740762376423, Probabilidad Acumulada: [0.8470284564775444]
Individuo: [10, 1, 9, 7, 8, 2, 4, 3], Funcion: 879, Probabilidad: 0.1529715435224555, Probabilidad Acumulada: [1.0]
Total Probabilidad: 1.0


## Selección

- El individuo con mayor función de calidad obtiene mayor probabilidad de ser seleccionado en el método de la ruleta (Se utiliza 1/f debido a que en el problema se está intentando minimizar)
- Se deben seleccionar 2 individuos para realizar mutación o cruce

In [41]:
def wheel_selection(poblacion):
    random_value = random.random()
    for ind in poblacion:
        if random_value <= ind.probabilidad_acumulada[-1]:
            return ind
    return poblacion[-1]

#### Valores a letras

In [19]:
def valores_a_letras(individuo_valores, fichas):
    valores_a_letras = {}
    for letra, valor in fichas.items():
        valores_a_letras.setdefault(valor, []).append(letra)

    resultado = []
    usadas = set()

    for i, val in enumerate(individuo_valores):
        posibles = valores_a_letras[val]

        if i == 0 and 10 in valores_a_letras and 'a' in posibles:
            resultado.append('a')
            usadas.add('a')
            continue
        elif i == len(individuo_valores) - 1 and 3 in valores_a_letras and 'j' in posibles:
            resultado.append('j')
            usadas.add('j')
            continue

        if val == 10:
            letra = 'h' if 'h' in posibles else posibles[0]
        elif val == 3:
            letra = 'b' if 'b' in posibles else posibles[0]
        else:
            # Buscar cualquier letra no usada
            letra = next((l for l in posibles if l not in usadas), posibles[0])
        
        resultado.append(letra)
        usadas.add(letra)

    return resultado

### Implementación

In [50]:
flag = True
while flag:
    individuo1 = wheel_selection(poblacion)
    individuo2 = wheel_selection(poblacion)
    if individuo1 == individuo2:
        print("Selected the same individual, re-selecting...")
        continue
    else:
        flag = False

print("Final Selected Individual 1:", valores_a_letras(individuo1.individuo, fichas), "with Probability:", individuo1.probabilidad)
print("Final Selected Individual 2:", valores_a_letras(individuo2.individuo, fichas), "with Probability:", individuo2.probabilidad)

Final Selected Individual 1: ['a', 'b', 'h', 'e', 'c', 'b', 'h', 'j'] with Probability: 0.06345377128055811
Final Selected Individual 2: ['a', 'b', 'f', 'c', 'b', 'd', 'c', 'j'] with Probability: 0.08294971260154119


## Cruces

In [51]:
def one_point_crossover(ind1, ind2):
    crossover_point = random.randint(1, len(ind1.individuo) - 2)
    print("Crossover Point:", crossover_point)
    child1 = ind1.individuo[:crossover_point] + ind2.individuo[crossover_point:]
    child2 = ind2.individuo[:crossover_point] + ind1.individuo[crossover_point:]
    return  child1, child2

child1, child2 = one_point_crossover(individuo1, individuo2)
print("Child 1:", valores_a_letras(child1, fichas))
print("Child 2:", valores_a_letras(child2, fichas))

Crossover Point: 2
Child 1: ['a', 'b', 'f', 'c', 'b', 'd', 'c', 'j']
Child 2: ['a', 'b', 'h', 'e', 'c', 'b', 'h', 'j']


In [56]:
def sorted_chromosomes(ind1, ind2):
    crossover_point = 4
    child1 = ind1.individuo[:crossover_point]
    child2 = ind2.individuo[:crossover_point]
    for i in range(len(ind2.individuo)):
        if ind2.individuo[i] in child1:
            continue
        else:
            child1.append(ind2.individuo[i])
    for i in range(len(ind1.individuo)):
        if ind1.individuo[i] in child2:
            continue
        else:
            child2.append(ind1.individuo[i])
    child1.append(ind1.individuo[-1])
    child2.append(ind2.individuo[-1])
    return child1, child2

child1, child2 = sorted_chromosomes(individuo1, individuo2)
print("Child 1:", valores_a_letras(child1, fichas))
print("Child 2:", valores_a_letras(child2, fichas))

Child 1: ['a', 'b', 'h', 'e', 'f', 'c', 'd', 'j']
Child 2: ['a', 'b', 'f', 'c', 'e', 'j']


## Mutaciones

## Construcción Modelo

In [43]:
solucion = []

for i in range(100):
    parents = []
    poblacion = creacion_poblacion(fichas, 10)
    probabilidades(poblacion)
    probabilidades_acum(poblacion)
    flag = True
    while flag:
        individuo1 = wheel_selection(poblacion)
        individuo2 = wheel_selection(poblacion)
        if individuo1 == individuo2:
            continue
        else:
            flag = False
            parents.append(individuo1)
            parents.append(individuo2)
    solucion.append(parents)

print("Final Selected Individuals:")
for i, (ind1, ind2) in enumerate(solucion): 
    print(f"Pair {i+1}:")
    print("  Individual 1:", valores_a_letras(ind1.individuo, fichas), "with Probability:", ind1.probabilidad)
    print("  Individual 2:", valores_a_letras(ind2.individuo, fichas), "with Probability:", ind2.probabilidad)

Final Selected Individuals:
Pair 1:
  Individual 1: ['a', 'b', 'g', 'd', 'h', 'b', 'g', 'j'] with Probability: 0.18507227416462976
  Individual 2: ['a', 'd', 'f', 'd', 'd', 'c', 'h', 'j'] with Probability: 0.17500924270886975
Pair 2:
  Individual 1: ['a', 'h', 'e', 'h', 'b', 'g', 'h', 'j'] with Probability: 0.08121998468681911
  Individual 2: ['a', 'e', 'f', 'b', 'c', 'd', 'c', 'j'] with Probability: 0.15188776663873652
Pair 3:
  Individual 1: ['a', 'e', 'h', 'b', 'b', 'b', 'f', 'j'] with Probability: 0.09477030154112712
  Individual 2: ['a', 'h', 'e', 'b', 'h', 'd', 'h', 'j'] with Probability: 0.08777904978809316
Pair 4:
  Individual 1: ['a', 'e', 'd', 'b', 'b', 'f', 'b', 'j'] with Probability: 0.24552495354859138
  Individual 2: ['a', 'd', 'h', 'c', 'g', 'b', 'g', 'j'] with Probability: 0.10741716717750874
Pair 5:
  Individual 1: ['a', 'h', 'g', 'h', 'f', 'c', 'd', 'j'] with Probability: 0.04138654147932782
  Individual 2: ['a', 'i', 'e', 'b', 'd', 'g', 'c', 'j'] with Probability: 0.

In [68]:
data = []
pares = 1

for i, (ind1, ind2) in enumerate(solucion):
    data.append({
        "Pair": pares,
        "Ind1Numerico": ind1.individuo,
        "Ind2Numerico": ind2.individuo,
        "Ind1Letras": valores_a_letras(ind1.individuo, fichas),
        "Ind2Letras": valores_a_letras(ind2.individuo, fichas),
        "Ind1Probabilidad": ind1.probabilidad,
        "Ind2Probabilidad": ind2.probabilidad
    })
    if i % 2 == 1:
        pares += 1

df_solucion = pd.DataFrame(data)

In [69]:
df_solucion

,Pair,Ind1Numerico,Ind2Numerico,Ind1Letras,Ind2Letras,Ind1Probabilidad,Ind2Probabilidad
0,1,"[10, 3, 4, 1, 10, 3, 4, 3]","[10, 1, 8, 1, 1, 7, 10, 3]","[a, b, g, d, h, b, g, j]","[a, d, f, d, d, c, h, j]",0.185072,0.175009
1,1,"[10, 10, 2, 10, 3, 4, 10, 3]","[10, 2, 8, 3, 7, 1, 7, 3]","[a, h, e, h, b, g, h, j]","[a, e, f, b, c, d, c, j]",0.081220,0.151888
2,2,"[10, 2, 10, 3, 3, 3, 8, 3]","[10, 10, 2, 3, 10, 1, 10, 3]","[a, e, h, b, b, b, f, j]","[a, h, e, b, h, d, h, j]",0.094770,0.087779
3,2,"[10, 2, 1, 3, 3, 8, 3, 3]","[10, 1, 10, 7, 4, 3, 4, 3]","[a, e, d, b, b, f, b, j]","[a, d, h, c, g, b, g, j]",0.245525,0.107417
4,3,"[10, 10, 4, 10, 8, 7, 1, 3]","[10, 9, 2, 3, 1, 4, 7, 3]","[a, h, g, h, f, c, d, j]","[a, i, e, b, d, g, c, j]",0.041387,0.162061
...,...,...,...,...,...,...,...
95,48,"[10, 3, 2, 3, 1, 4, 3, 3]","[10, 8, 10, 1, 3, 2, 9, 3]","[a, b, e, b, d, g, b, j]","[a, f, h, d, b, e, i, j]",0.395169,0.063940
96,49,"[10, 4, 8, 4, 1, 9, 2, 3]","[10, 1, 7, 4, 1, 2, 7, 3]","[a, g, f, g, d, i, e, j]","[a, d, c, g, d, e, c, j]",0.104596,0.300063
97,49,"[10, 1, 10, 2, 10, 3, 4, 3]","[10, 9, 4, 2, 2, 9, 4, 3]","[a, d, h, e, h, b, g, j]","[a, i, g, e, e, i, g, j]",0.194740,0.141860
98,50,"[10, 4, 3, 7, 3, 1, 10, 3]","[10, 2, 10, 2, 1, 1, 1, 3]","[a, g, b, c, b, d, h, j]","[a, e, h, e, d, d, d, j]",0.134269,0.194202
